In [ ]:
using WAV;
using Gen;
import Random;
include("rendering.jl")
include("model.jl");
include("plotting.jl");
include("proposals.jl");
include("gammatonegram.jl");
include("time_helpers.jl");
include("inference_helpers.jl");
include("initialization.jl");

In [ ]:
function harmonic_source(erbf0, filt, n_harmonics, duration, steps, sr, rms_ref)
    
    log_const = 1e-12; #cell_mean=0.0
    
    ##Preparing windows & filters; prepare latents
    #Latent variables resize for edges of element
    erbf0 = vcat([erbf0[1]], erbf0, [erbf0[end]])
    filt = vcat( reshape(filt[1,:],(1,size(filt)[2])), filt, reshape(filt[end,:],(1,size(filt)[2])))
    #Generate window & filter
    win_arr, sig_len = make_overlapping_time_windows(size(filt)[1], steps["t"], audio_sr) 
    wlen, n_windows = size(win_arr)
    filterbank, freq_cutoffs = make_overlapping_freq_windows(size(filt)[1], steps, audio_sr) 
    flen, n_channels = size(filterbank)
    if wlen == flen
        sig_len = wlen
    else
        error("Incorrect array lengths")
    end
    #Convert spectrum level into energy
    bandwidthsHz = repeat(reshape([Hz[2] - Hz[1] for Hz in freq_cutoffs[end:-1:1]], 1, n_channels),n_windows,1) 
    energy_grid = transpose( rms_ref*10.0 .^( (filt + 10*log.(10, bandwidthsHz))/20.0 ) .- log_const)
    
    ##Generating harmonic stack source - frequency_modulation
    hi_lim_freq = sr/2. - 1;
    hi_lim_ERB = freq_to_ERB(floor(hi_lim_freq))
    f0 = ERB_to_freq(erbf0)
    source = zeros(Int(round(sig_len)))
    for i = 1:n_harmonics
        f = f0 .* i
        if all(f .> hi_lim_freq)
            break
        end
        f = [f_ < 20.0 ? 20.0 : f_ for f_ in f]
        erbf = freq_to_ERB(f)
        h, unclamped_f = FM_excitation(erbf, sig_len, steps["t"], sr)
        too_high = unclamped_f .> hi_lim_freq
        for j = 1:length(too_high)
            h[j] = too_high[j] ? 0.0 : h[j]
        end
        source += h
    end
    #Make the source pink
    fft_sig = FFTW.dct(source)
    binFactor = float(sig_len/sr)
    pinkWeights = binFactor*range(1,stop=sig_len)
    fft_sig = fft_sig ./ sqrt.(pinkWeights)
    source = FFTW.idct(fft_sig)
    
    #amplitude modulation 
    AM_FM_tone = zeros(Int(round(sig_len)))
    nonzero(a) = a != 0.
    for curr_win_i = 1:n_windows
        
        #section of source to modulate
        curr_win = win_arr[:,curr_win_i] 
        indices = findall(nonzero, curr_win)
        windowed_source = source.*curr_win
        noise_rms = sqrt(mean((windowed_source[indices]).^2.)) ##??
        norm_windowed_source = windowed_source ./ noise_rms
        
        #preparing filter
        e_i = energy_grid[:, curr_win_i] #n_channels
        e_i = reshape(e_i, (1, n_channels))
        e_i = repeat(e_i, sig_len, 1)
        weighted_filterbank = e_i .* filterbank
        filter = sum(weighted_filterbank, dims=2)
        
        #applying filter
        fft_sig = FFTW.dct(norm_windowed_source)
        filtered_norm_windowed_source = FFTW.idct(filter .* fft_sig)
        AM_FM_tone += filtered_norm_windowed_source
        
    end
    
    n_samples = Int(floor(sr*duration))
    start_point = max(1, Int(floor((length(AM_FM_tone) - n_samples)/2)))
    AM_FM_tone = AM_FM_tone[start_point:start_point+n_samples-1]
    
    #Apply ramps
    AM_FM_tone = hann_ramp(AM_FM_tone[:,1],sr)
    
    return AM_FM_tone
    
    end;

In [ ]:
@gen function sample_harmonic_gp(element_timing, gp_params, steps, audio_sr, rms_ref)
    ## Define points at which the GPs should be sampled
    gps = Dict()
    gps[:amp], gps[:erb], fs = get_gp_spectrotemporal(element_timing, steps, audio_sr) 
    ## Sample the GPs in a sequential manner
    features = [:erb, :amp]
    for feature in features
        mu, cov = get_mu_cov(gps[feature], gp_params[feature]) 
        gps[feature] = @trace(mvnormal(mu, cov), (:element, 1) => feature)
    end
    
    ## Generate the waveform given these element GPs
    element_duration = element_timing[2] - element_timing[1];
    amp = transpose(reshape(gps[:amp], (length(fs), length(gps[:erb]))))
    println(amp)
    element_wave = harmonic_source(gps[:erb], amp, 40, element_duration, steps, audio_sr, rms_ref)
        
    return element_wave
    
end

In [ ]:
@gen function sample_harmonic(duration, steps, audio_sr, wts, gtg_params)

    gp_params = Dict(:erb => Dict(), :amp => Dict())
    
    #fundamental frequency
    gp_params[:erb][:mu] = @trace(uniform(freq_to_ERB(20.0), freq_to_ERB(1000.0)), :erb => :mu)
    gp_params[:erb][:scale] = @trace(gamma(0.5,1), :erb => :scale) 
    gp_params[:erb][:sigma] = @trace(gamma(3,1), :erb => :sigma) 
    gp_params[:erb][:noise] = @trace(gamma(1,1), :erb => :noise) 

    #amplitude
    gp_params[:amp][:mu] = @trace(normal(15, 10), :amp => :mu) #spectrum level!
    gp_params[:amp][:scale_t] = @trace(gamma(0.5,1), :amp => :scale_t) 
    gp_params[:amp][:scale_f] = @trace(gamma(1,1), :amp => :scale_f) 
    gp_params[:amp][:sigma] = @trace(gamma(10,1), :amp => :sigma) 
    gp_params[:amp][:noise] = @trace(gamma(1,1), :amp => :noise) 
    
    harmonic = sample_harmonic_gp([0,duration], gp_params, steps, audio_sr, gtg_params["ref"])
    gtg, t = gammatonegram(harmonic, wts, audio_sr, gtg_params)
    
    return gtg, harmonic

    end;

In [ ]:
audio_sr=20000; steps = Dict("t"=>0.020, "min"=> 0.020, "f"=>4); 
gtg_params=Dict("twin"=>0.025, "thop"=>0.010, "nfilts"=>64, "fmin"=>50, "width"=>0.50, 
    "log_constant"=>1e-80, "dB_threshold"=>20, "ref"=>1e-6, "plot_max"=>90)
wts, f = gtg_weights(audio_sr, gtg_params)
duration = 0.500;

In [ ]:
args = (duration, steps, audio_sr, wts, gtg_params)
trace, = generate(sample_harmonic, args)

In [ ]:
gtg, wave = get_retval(trace)
plot(wave)

In [ ]:
plot_gtg(gtg, duration, audio_sr/2., 0, 100)

In [ ]:
wavwrite(wave/maximum(abs.(wave)), "harmonic.wav", Fs=audio_sr)